In [1]:
from flask import Flask, request, jsonify
import time
import os
import azure.cognitiveservices.speech as speechsdk
import json 
from datetime import datetime 
import subprocess
from pydub.utils import mediainfo
import json
import cv2
import os, shutil
import boto3
from io import StringIO
import sys
import time
from moviepy.editor import VideoFileClip
import threading

import torch
# from memory_profiler import profile
# import gc
# import matplotlib.pyplot as plt
from PIL import Image
import cv2
import torch.hub
import time

# from flask import Flask, request, jsonify
# import os
# import shutil
# from moviepy.editor import VideoFileClip

In [2]:
def convert_video_to_wav(video_path):
    video_data = mediainfo(video_path)
    channels = video_data["channels"]
    bit_rate = video_data["bit_rate"]
    sample_rate = video_data["sample_rate"]
    wav_output = video_path.replace(".mp4",".wav").replace("video","wav_file")

    ffmpeg_cmd = ['ffmpeg', '-i', video_path,  '-b:a', bit_rate, '-ar', sample_rate , '-ac', channels ,'-vn', wav_output]
    
    try:
        subprocess.check_output(ffmpeg_cmd, stderr=subprocess.STDOUT)
        print("Video converted to WAV successfully!")
        return wav_output
    except subprocess.CalledProcessError as e:
        print("Error converting video to WAV:", e.output)
        return None


In [3]:
# def transcribe(key,region,lang,path_in,path_out="out.txt",newLine=False):auto_detect_source_language_config
def transcribe(key,region,auto_detect_source_language_config,path_in,path_out="out.txt",newLine=False):
    # lang = auto_detect_source_language_config
    speech_config = speechsdk.SpeechConfig(subscription=key, region=region)
    # speech_config.speech_recognition_language=lang
    speech_config.output_format = speechsdk.OutputFormat(1)
    speech_config.request_word_level_timestamps()
    audio_config = speechsdk.AudioConfig(filename=path_in)
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config,auto_detect_source_language_config=auto_detect_source_language_config, audio_config=audio_config)
    done = False
    tep_text_json = []
    textOut = ""
    def stop_cb(evt):
        print(1)
        print(evt)
        speech_recognizer.stop_continuous_recognition()
        nonlocal done
        done = True

    str_newLine = ""
    if newLine:
        str_newLine = " \n"

    def outPrint(evt):
        nonlocal textOut
        nonlocal tep_text_json
        tmp_text = evt.result.text
        tmp_timestamp = evt.result.offset
        tmp_duration_ts = evt.result.duration
        textOut += f"[{tmp_timestamp}] [{tmp_duration_ts}] {tmp_text}" + str_newLine
        
        out = eval(evt.result.json)
        tep_text_json.append(out)
        # print(2)
        # print(tmp_text)
        # print(3)
        # print(tmp_timestamp)
        # print(4)
        # print(tmp_duration_ts)
        # print(5)
        # print('-' * 100)
        # print(evt)
        # print('-' * 100)
        # print(evt.result)
        # print('-' * 100)
        # print("haa yahi h be")
        # print(evt.result.json)
        # out = eval(evt.result.json)
        # print(type(out))
        # print(out)
        # print('-' * 100)
        # print(evt.result.properties)
        # print('-' * 100)
        # print(dir(evt))
        # print('-' * 100)
        # print(dir(evt.result))
    
    speech_recognizer.recognized.connect(outPrint)
    speech_recognizer.session_stopped.connect(stop_cb)
    speech_recognizer.start_continuous_recognition()

    while not done:
        time.sleep(.5)
    # with open(path_out, 'w', encoding='utf-16') as f:
    #     f.write(textOut)
    # tep_text_json = str(tep_text_json)
    # path_out="./"
    print("tep_text_json:",tep_text_json)
    json_data = json.dumps(tep_text_json)
    print("json_data:",json_data)
    with open(path_out, 'w', encoding='utf-8') as f:
        f.write(json_data)

In [4]:
def milliseconds_to_srt_timestamp(milliseconds):
    total_seconds = milliseconds // 1000
    # milliseconds %= 1000
    # minutes = seconds // 60
    # seconds %= 60
    hours = total_seconds // 3600
    minutes = (total_seconds % 3600) // 60
    seconds = total_seconds % 60
    milliseconds = milliseconds % 1000
    # minutes %= 60
    return '{:02d}:{:02d}:{:02d},{:03d}'.format(hours, minutes, seconds, milliseconds)


# json_file = "D:/Python_scripts/speech_to_test_azure/New folder/txt_&_json_file/txt_&_json_file/Piyush_2.json"  # Replace with the path to your JSON file
def convert_json_into_words_by_words_list(json_file):
    print("adsefesdwefewdwefwdwedbadja bseujwsduwbdakuscbasukdasbdaeudaeudavsxuasvbaoudygwuoqevwqb8dyasdwvwe")
    with open(json_file, "r",encoding='utf-8') as file:
        json_output = file.read()

    data = json.loads(json_output)
    # print(len(data))
    print(data)
    srt_content = ""
    list_of_words = []
    for j in range(len(data)):
        # print(j)
    # Extract word-level details
        try:
            words = data[j]["NBest"][0]["Words"]

        # Generate SRT content
            for i, word in enumerate(words):
                list_with_time =[]
                start_timestamp = milliseconds_to_srt_timestamp(word["Offset"] // 10000)
                end_timestamp = milliseconds_to_srt_timestamp((word["Offset"]  + word["Duration"]) // 10000)
                subtitle_text = word["Word"]
                srt_entry = f"{i+1}\n{start_timestamp} --> {end_timestamp}\n{subtitle_text}\n\n"
                srt_content += srt_entry
                list_with_time.append(start_timestamp)
                list_with_time.append(end_timestamp)
                list_with_time.append(subtitle_text)
                list_of_words.append(list_with_time)
        except:
            pass
    # json_data = json.dumps(list_of_words)
    print("json_data "*80)
    # json_data = json.loads(json_data)
    with open("./list_of_words.json","w",encoding="utf-8") as f:
        json.dump(list_of_words,f,ensure_ascii=False)
    return list_of_words

# json_file = "D:/Python_scripts/speech_to_test_azure/srt_fol/pm_modi.json"

# list_of_words = convert_json_into_words_by_words_list(json_file)

In [5]:

def convert_time_stamp_to_seconds(time_stamp):
    time_format = "%H:%M:%S,%f"

    # Convert the time stamp to a datetime object
    time_obj = datetime.strptime(time_stamp, time_format)

    # Calculate the total number of seconds
    seconds = time_obj.hour * 3600 + time_obj.minute * 60 + time_obj.second + time_obj.microsecond / 1000000

    return seconds

In [6]:
def generate_srt_file(list_of_words,filename):

    list_of_new_srt = []
    str_of_words = ""
    total_words_covered = 0
    index = 1
    srt_sentence = ""

    list_for_sentences_for_the_final_srt =[]
    while total_words_covered != len(list_of_words):
        transcription_list = []
        start_point = convert_time_stamp_to_seconds(list_of_words[total_words_covered][0])
        start_for_srt = list_of_words[total_words_covered][0]

        transcription_list.append(list_of_words[total_words_covered][2])
        total_words_covered += 1

        
        list_for_srt_by_sentence = []
        
        dict = {}

        # DECIDE AN END FOR START WORD

        for j in range(total_words_covered, (len(list_of_words) - 1)):

            end_point =convert_time_stamp_to_seconds(list_of_words[j][1])
            end_for_srt = list_of_words[j][1]
            end_for_prev_srt = list_of_words[j - 1][1]
        
            time_limit = round(((end_point) - (start_point)),3)
            
            
            # bin_size
            if time_limit <= (2.50):
                transcription_list.append(list_of_words[j][2])
                total_words_covered += 1
                end_for_srt = list_of_words[j][1]

            else:
                list_for_srt_by_sentence.append(transcription_list)
                dict["transcript"] = transcription_list
                dict["start_point"] = start_for_srt
                dict["end_point"]  = end_for_prev_srt
                break
    

        list_of_new_srt.append(dict)
        str_of_words = ""
    for transcript_dict in list_of_new_srt:
        dict ={}
        if "transcript" in transcript_dict.keys():
            subtitle_text = (' '.join(transcript_dict["transcript"]))
            start_time = transcript_dict["start_point"]
            end_time = transcript_dict["end_point"] 
            dict["transcript"] = subtitle_text
            dict["start_point"] = start_time
            dict["end_point"] = end_time
            list_for_sentences_for_the_final_srt.append(dict)

    for dict_sentences in list_for_sentences_for_the_final_srt:
        # print(dict_sentences)
        subtitle_text = dict_sentences["transcript"]
        start_timestamp = dict_sentences['start_point']
        end_timestamp = dict_sentences['end_point']
        srt_entry = f"{index}\n{start_timestamp} --> {end_timestamp}\n{subtitle_text}\n\n"
        index +=1
        # print(srt_entry)
        # break
        srt_sentence += srt_entry

        # srt_content += ""+ srt_entry

    filename = filename

    with open(filename, 'w',encoding="utf-8") as file:
        file.write(srt_sentence)
    return srt_sentence

    
    

In [8]:

class VideoDetect:
    jobId = ''
    rek = boto3.client('rekognition', region_name='ap-south-1')
    sqs = boto3.client('sqs', region_name='ap-south-1')
    sns = boto3.client('sns', region_name='ap-south-1')

    roleArn = ''
    bucket = ''
    video = ''
    startJobId = ''

    sqsQueueUrl = ''
    snsTopicArn = ''
    processType = ''

    def __init__(self, role, bucket, video, segment):
        self.roleArn = role
        self.bucket = bucket
        self.video = video
        self.segment = segment


    def GetSQSMessageSuccess(self):

        jobFound = False
        succeeded = False

        dotLine=0
        while jobFound == False:
            sqsResponse = self.sqs.receive_message(QueueUrl=self.sqsQueueUrl, MessageAttributeNames=['ALL'],
                                                   MaxNumberOfMessages=10)

            if sqsResponse:

                if 'Messages' not in sqsResponse:
                    if dotLine<40:
                        print('.', end='')
                        dotLine=dotLine+1
                    else:
                        print()
                        dotLine=0
                    sys.stdout.flush()
                    time.sleep(5)
                    continue

                for message in sqsResponse['Messages']:
                    notification = json.loads(message['Body'])
                    rekMessage = json.loads(notification['Message'])
                    print(rekMessage['JobId'])
                    print(rekMessage['Status'])
                    if rekMessage['JobId'] == self.startJobId:
                        # print('Matching Job Found:' + rekMessage['JobId'])
                        jobFound = True
                        if (rekMessage['Status']=='SUCCEEDED'):
                            succeeded=True

                        self.sqs.delete_message(QueueUrl=self.sqsQueueUrl,
                                                ReceiptHandle=message['ReceiptHandle'])
                    else:
                        pass
                        # print("Job didn't match:" +
                        #       str(rekMessage['JobId']) + ' : ' + self.startJobId)
                    # Delete the unknown message. Consider sending to dead letter queue
                    self.sqs.delete_message(QueueUrl=self.sqsQueueUrl,
                                            ReceiptHandle=message['ReceiptHandle'])


        return succeeded

    # def StartCelebrityRecognition(self):
    def StartContentModeration(self):
        # response=self.rek.start_label_detection(Video={'S3Object': {'Bucket': self.bucket, 'Name': self.video}},
        #                                         NotificationChannel={'RoleArn': self.roleArn, 'SNSTopicArn': self.snsTopicArn})
        # response=self.rek.start_celebrity_recognition(Video={'S3Object': {'Bucket': self.bucket, 'Name': self.video}},
        print(self.bucket)
        print(self.video)
        response=self.rek.start_content_moderation(Video={'S3Object': {'Bucket': self.bucket, 'Name': self.video}},
                                                   NotificationChannel={'RoleArn': self.roleArn, 'SNSTopicArn': self.snsTopicArn})

        self.startJobId=response['JobId']
        # print('Start Job Id: ' + self.startJobId)
    def StartCelebrityRecognition(self):
        # response=self.rek.start_label_detection(Video={'S3Object': {'Bucket': self.bucket, 'Name': self.video}},
        #                                         NotificationChannel={'RoleArn': self.roleArn, 'SNSTopicArn': self.snsTopicArn})
        # response=self.rek.start_celebrity_recognition(Video={'S3Object': {'Bucket': self.bucket, 'Name': self.video}},
        print(self.bucket)
        print(self.video)
        response=self.rek.start_celebrity_recognition(Video={'S3Object': {'Bucket': self.bucket, 'Name': self.video}},
                                                   NotificationChannel={'RoleArn': self.roleArn, 'SNSTopicArn': self.snsTopicArn})

        self.startJobId=response['JobId']
        # print('Start Job Id: ' + self.startJobId)


    # def GetCelebrityRecognitionResults(self):
    def GetContentModerationResults(self):
        maxResults = 100
        paginationToken = ''
        # finished = False
        #
        # while finished == False:
        response = self.rek.get_content_moderation(JobId=self.startJobId,
                                                   MaxResults=maxResults,
                                                   NextToken=paginationToken,
                                                   SortBy='TIMESTAMP')

        print('Codec: ' + response['VideoMetadata']['Codec'])
        # print('Duration: ' + str(response['VideoMetadata']['DurationMillis']))
        print('Format: ' + response['VideoMetadata']['Format'])
        print('Frame rate: ' + str(response['VideoMetadata']['FrameRate']))
        print()

        result_list = []

        for moderation_label_info in response['ModerationLabels']:
            moderation_label_dict = {}
            segment_offset_seconds = (self.segment - 1) * 60000
            timestamp = moderation_label_info["Timestamp"]
            timestamp += segment_offset_seconds
            label_name = moderation_label_info["ModerationLabel"]["Name"]
            confidence = moderation_label_info["ModerationLabel"]["Confidence"]

            milliseconds = timestamp
            seconds,milliseconds = divmod(milliseconds,1000)
            minutes,seconds = divmod(seconds,60)
            hours, minutes = divmod(minutes,60)
            timestamp_proper_format = f"{hours:02d}:{minutes:02d}:{seconds:02d}"
            moderation_label_dict['timestamp_in_Hours_min_sec'] = timestamp_proper_format
            moderation_label_dict['timestamp'] = timestamp
            moderation_label_dict['Moderated Label Name'] = label_name
            moderation_label_dict['Confidence'] = confidence

            result_list.append(moderation_label_dict)

            print('Timestamp: ', timestamp, '\tModerated Label: ', label_name, "\t Confidence: ", confidence)
        return result_list

    def GetCelebrityRekognitionResults(self):
        maxResults = 100
        paginationToken = ''
        # finished = False
        #
        # while finished == False:
        response = self.rek.get_celebrity_recognition(JobId=self.startJobId,
                                                   MaxResults=maxResults,
                                                   NextToken=paginationToken,
                                                   SortBy='TIMESTAMP')

        print('Codec: ' + response['VideoMetadata']['Codec'])
        # print('Duration: ' + str(response['VideoMetadata']['DurationMillis']))
        print('Format: ' + response['VideoMetadata']['Format'])
        print('Frame rate: ' + str(response['VideoMetadata']['FrameRate']))
    
        # print()

        result_list = []

        for celeb_name in response['Celebrities']:
            celeb_dict = {}
            segment_offset_seconds = (self.segment - 1) * 60000
            timestamp = celeb_name["Timestamp"]
            timestamp += segment_offset_seconds
            name = celeb_name["Celebrity"]["Name"]
            confidence = celeb_name["Celebrity"]["Confidence"]

            milliseconds = timestamp
            seconds,milliseconds = divmod(milliseconds,1000)
            minutes,seconds = divmod(seconds,60)
            hours, minutes = divmod(minutes,60)
            timestamp_proper_format = f"{hours:02d}:{minutes:02d}:{seconds:02d}"

            celeb_dict['timestamp_in_proper_format'] = timestamp_proper_format
            celeb_dict['timestamp'] = timestamp
            celeb_dict['Celeb Name'] = name
            celeb_dict['Confidence'] = confidence

            result_list.append(celeb_dict)

            # print('Timestamp: ', timestamp, '\tModerated Label: ', label_name, "\t Confidence: ", confidence)
        return result_list



    def CreateTopicandQueue(self):

        millis = str(int(round(time.time() * 1000)))

        #Create SNS topic

        snsTopicName="AmazonRekognitionExample" + millis

        topicResponse=self.sns.create_topic(Name=snsTopicName)
        self.snsTopicArn = topicResponse['TopicArn']

        #create SQS queue
        sqsQueueName="AmazonRekognitionQueue" + millis
        self.sqs.create_queue(QueueName=sqsQueueName)
        self.sqsQueueUrl = self.sqs.get_queue_url(QueueName=sqsQueueName)['QueueUrl']

        attribs = self.sqs.get_queue_attributes(QueueUrl=self.sqsQueueUrl,
                                                AttributeNames=['QueueArn'])['Attributes']

        sqsQueueArn = attribs['QueueArn']

        # Subscribe SQS queue to SNS topic
        self.sns.subscribe(
            TopicArn=self.snsTopicArn,
            Protocol='sqs',
            Endpoint=sqsQueueArn)

        #Authorize SNS to write SQS queue
        policy = """{{
  "Version":"2012-10-17",
  "Statement":[
    {{
      "Sid":"MyPolicy",
      "Effect":"Allow",
      "Principal" : {{"AWS" : "*"}},
      "Action":"SQS:SendMessage",
      "Resource": "{}",
      "Condition":{{
        "ArnEquals":{{
          "aws:SourceArn": "{}"
        }}
      }}
    }}
  ]
}}""".format(sqsQueueArn, self.snsTopicArn)

        response = self.sqs.set_queue_attributes(
            QueueUrl = self.sqsQueueUrl,
            Attributes = {
                'Policy' : policy
            })

    def DeleteTopicandQueue(self):
        self.sqs.delete_queue(QueueUrl=self.sqsQueueUrl)
        self.sns.delete_topic(TopicArn=self.snsTopicArn)


def delete_temp_file(file_path):
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))


In [9]:
import re
def extract_segment_number(filename):
    pattern = r'_segment_(\d+)\.mp4'
    match = re.search(pattern, filename)
    if match:
        segment_number = int(match.group(1))
        return segment_number
    else:
        return None

In [10]:
def split_video_into_segments(input_file, segment_duration=60):
    try:
        clip = VideoFileClip(input_file)

        # Calculate the total duration of the input video in seconds
        total_duration = clip.duration

        # Calculate the number of segments required
        num_segments = int(total_duration / segment_duration)

        # Extract the file name and extension from the input_file
        file_name, file_extension = input_file.split('.')

        # Split the video into segments
        for i in range(num_segments):
            start_time = i * segment_duration
            end_time = min((i + 1) * segment_duration, total_duration)
            segment_clip = clip.subclip(start_time, end_time)
            output_file = f"{file_name}_segment_{i + 1}.{file_extension}"
            segment_clip.write_videofile(output_file, codec="libx264")

        # For the last segment (if it's shorter than segment_duration)
        if total_duration % segment_duration > 0:
            start_time = num_segments * segment_duration
            end_time = total_duration
            last_segment_clip = clip.subclip(start_time, end_time)
            output_file = f"{file_name}_segment_{num_segments + 1}.{file_extension}"
            last_segment_clip.write_videofile(output_file, codec="libx264")

        clip.close()
        print("Video splitting completed successfully!")
    except Exception as e:
        print(f"An error occurred: {str(e)}")

In [11]:
def split_video_and_upload_s3(video_path):
    cwd = os.getcwd()
    listdir_earlier = os.listdir(cwd)
    print(listdir_earlier)
    # ffmpeg_cmd = ['ffmpeg', '-i', video_path,  '-c', 'copy', '-map 0', '-segment_time' , '00:01:00', "-f segment -reset_timestamps 1 output%03d.mp4"]
    # ffmpeg_split_cmd = """ffmpeg -i '""" + video_path + """' -c copy -map 0 -segment_time 00:01:00 -f segment -reset_timestamps 1 output%03d.mp4"""
    ffmpeg_split_cmd = """ffmpeg -i 'video/THOMAS_SHELBY_SMOKING.mp4' -c copy -map 0 -segment_time 00:01:00 -f segment -reset_timestamps 1 output%03d.mp4 > /dump 2>&1"""
    # subprocess.check_output(ffmpeg_cmd, stderr=subprocess.STDOUT)
    # subprocess.run(ffmpeg_split_cmd, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    try:
        print(ffmpeg_split_cmd)
        subprocess.check_output(ffmpeg_split_cmd, shell=True, stderr=subprocess.STDOUT)
    except subprocess.CalledProcessError as e:
        raise RuntimeError("command '{}' return with error (code {}): {}".format(e.cmd, e.returncode, e.output))

    listdir_now = os.listdir(cwd)
    print(listdir_now)

    # cmd = 'aws s3 cp D:/Python_scripts/AWS_Scripts/my_function/my_deployment_package.zip s3://alt-cap-logsbucket-12leb47kv5fyr/'

In [12]:
def upload_to_s3(video_path):
    cmd = 'aws s3 cp "' + video_path + '" s3://aws-rekognition-dataset-v2'
    # subprocess.run(cmd, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    try:
        subprocess.check_output(cmd, shell=True, stderr=subprocess.STDOUT)
    except subprocess.CalledProcessError as e:
        raise RuntimeError("command '{}' return with error (code {}): {}".format(e.cmd, e.returncode, e.output))
    finally:
            delete_temp_file(video_path)
    

In [13]:
def get_videos_with_nomenclature():
    eligible_videos = []
    for filename in os.listdir('D:/Demo_Resources/Moderation_AI/video'):
        if filename.endswith('.mp4') and '_segment_' in filename:
            filename = 'D:/Demo_Resources/Moderation_AI/video/' + filename
            eligible_videos.append(filename)
            upload_to_s3(filename)  # Upload the video to S3 (you need to implement this function)
    return eligible_videos

In [14]:
def testing_thread_1(list_video_segments):
    # try:
        print("list_video_segments "*80)
        print(list_video_segments)
        save_path = 'D:/Demo_Resources/Moderation_AI/video/'
        # json_path = 'D:/Demo_Resources/Transcription_AI/Transcript_flask/json_file/'
        key = "<Azure key>"
        region = "centralindia"
        lang = "en-IN"
        # split_video_into_segments(path_of_video)
        # list_video_segments = get_videos_with_nomenclature()
        final_output_dict = {}
        final_output_of_cele_rec = []
        final_output_of_content_mode = []

        for video_segment in list_video_segments:
            # upload_to_s3(path_of_video)

            roleArn = 'arn:aws:iam::<account_id>:role/AmazonRekognitionVideoServiceRole'
            bucket = '<aws bucket>'
            # UNCOMMENT THIS FOR SHORT VIDEO
            # video = video_file.filename
            video = video_segment
            video = video.split('/')[-1]
            segment = extract_segment_number(video)

            print('Content Moderation Summary:')

            analyzer = VideoDetect(roleArn, bucket, video,segment)
            analyzer.CreateTopicandQueue()

            output_value_of_srt = None

            analyzer.StartCelebrityRecognition()
            for i in range(1):
                if analyzer.GetSQSMessageSuccess()==True:
                    output_of_Celebrity_rec=analyzer.GetCelebrityRekognitionResults()
                    # output_of_Content_moderation = analyzer.GetContentModerationResults()
                    print('Right before break!!')
                    # final_output_of_content_mode.append(output_of_Content_moderation)
                    final_output_of_cele_rec.append(output_of_Celebrity_rec)
                    break
            analyzer.DeleteTopicandQueue()
            print(final_output_of_cele_rec)
        # for video_segment in list_video_segments:
        #     delete_temp_file(video_segment)
        # final_output_dict["cele_output"] = final_output_of_cele_rec
        # final_output_dict["Content_mode"] = final_output_of_content_mode
        
        return final_output_of_cele_rec,200
    # except Exception as e:
    #     # Log the exception for debugging purposes
    #     print(f"Exception occurred: {e}")
    #     return str(e), 500


In [15]:
# testing_thread_1(['D:/Demo_Resources/Moderation_AI/video/Patal_lok_segment_1.mp4', 'D:/Demo_Resources/Moderation_AI/video/Patal_lok_segment_2.mp4', 'D:/Demo_Resources/Moderation_AI/video/Patal_lok_segment_3.mp4', 'D:/Demo_Resources/Moderation_AI/video/Patal_lok_segment_4.mp4', 'D:/Demo_Resources/Moderation_AI/video/Patal_lok_segment_5.mp4']
# )

In [16]:
def testing_thread_2(list_video_segments):
    try:
        print("list_video_segments "*80)
        print(list_video_segments)
        save_path = 'D:/Demo_Resources/Moderation_AI/video/'
        # json_path = 'D:/Demo_Resources/Transcription_AI/Transcript_flask/json_file/'
        key = "<Azure key>"
        region = "centralindia"
        lang = "en-IN"
        # split_video_into_segments(path_of_video)
        # list_video_segments = get_videos_with_nomenclature()
        # final_output_dict = {}
        # final_output_of_cele_rec = []
        final_output_of_content_mode = []

        for video_segment in list_video_segments:
            # upload_to_s3(path_of_video)

            roleArn = 'arn:aws:iam::<AWS_ID>:role/AmazonRekognitionVideoServiceRole'
            bucket = '<aws bucket>'
            # UNCOMMENT THIS FOR SHORT VIDEO
            # video = video_file.filename
            video = video_segment
            video = video.split('/')[-1]
            segment = extract_segment_number(video)


            print('Content Moderation Summary:')

            analyzer = VideoDetect(roleArn, bucket, video,segment)
            analyzer.CreateTopicandQueue()

            output_value_of_srt = None
            analyzer.StartContentModeration()
            for i in range(1):
                if analyzer.GetSQSMessageSuccess()==True:
                    # output_of_Celebrity_rec=analyzer.GetCelebrityRekognitionResults()
                    output_of_Content_moderation = analyzer.GetContentModerationResults()
                    print('Right before break!!')
                    final_output_of_content_mode.append(output_of_Content_moderation)
                    # final_output_of_cele_rec.append(output_of_Celebrity_rec)
                    break
            analyzer.DeleteTopicandQueue()
            print("final_output_of_content_mode "*80)
            print(final_output_of_content_mode)
        # for video_segment in list_video_segments:
        #     delete_temp_file(video_segment)
        # final_output_dict["cele_output"] = final_output_of_cele_rec
        # final_output_dict["Content_mode"] = final_output_of_content_mode
        
        return final_output_of_content_mode,200
    except Exception as e:
        # Log the exception for debugging purposes
        print(f"Exception occurred: {e}")
        return str(e), 500



In [17]:
def testing_thread_3(video_file):
    try:
        video_file = video_file
        
        save_path = 'D:/Demo_Resources/Moderation_AI/video/'
        json_path = 'D:/Demo_Resources/Moderation_AI/json_file/'
        
        path_of_video = (save_path + video_file.filename)
        key = "<Azure key>"
        region = "centralindia"
        auto_detect_source_language_config = speechsdk.languageconfig.AutoDetectSourceLanguageConfig(
        languages=["en-IN", "hi-IN",])
        path_of_wav_file = convert_video_to_wav(path_of_video)
        path_of_json = (json_path + video_file.filename)
        path_of_json = path_of_json.replace(".mp4",".json")
        print("path_of_json "*80)
        print(path_of_json)
        transcribe(key,region,auto_detect_source_language_config,path_of_wav_file,path_of_json)
        
        
        list_of_words = convert_json_into_words_by_words_list(path_of_json)
        print("list_of_words "*80)
        print(list_of_words)
        path_of_srt = path_of_video.replace("video","srt_file").replace(".mp4",".srt")
        print("path_of_srt "*80)
        print(path_of_srt)
        output_value_of_srt = generate_srt_file(list_of_words,path_of_srt)

        print(path_of_json)
        print("path_of_wav_file:",path_of_wav_file)
        return output_value_of_srt, 200
    except Exception as e:
        # Log the exception for debugging purposes
        print(f"Exception occurred: {e}")
        return str(e), 500


In [41]:
def testing_thread_4(video_path):
    
    classes_monuments = ['1000 Pillar Temple Moodabidri', 'Aaliyar Dam Park', 'Academy of Fine Arts Kolkata', 'Achalgarh Fort', 'Achut Bibis Mosque', 'Adalaj Stepwell', 'Adam Fountain', 'Adhanur temple', 'Adhi Ratneswarar Temple', 'Adina Mosque', 'Adinath Temple', 'Adiyogi Shiva statue', 'Afghan Church', 'Aga Khan Palace', 'Agra Fort', 'Agrasen Ki Baoli', 'Ahinsa Sthal', 'Aina Mahal', 'Airavateswarar Temple', 'Ajanta Caves', 'Akal Takht Amritsar', 'Akbar Tomb', 'Akshardham temple', 'Alamgir Masjid', 'Albert Hall Museum', 'Alipore Zoological Gardens', 'All Saints, Cathedral', 'Allahabad Fort', 'Allahabad Public Library', 'Alwar Qila', 'Amarnath Temple', 'Ambalappuzha Sri Krishna Temple', 'Ambedkar Park', 'Amber fort', 'Amer Fort', 'Amruteshwar Temple', 'Ananta Basudeba Temple', 'Ananta Vasudev Temple', 'Ananthapura Lake Temple', 'Anna Memorial', 'Antillia', 'Armenian Church', 'Arthanareeswarar Temple', 'Arulmigu Arunachaleswarar Temple', 'Ashtalakshmi Temple', 'Asirgarh Fort', 'Athirappilly Falls', 'Bada Bagh', 'Bada Gumbad', 'Bada Imambara', 'Badami Cave Temples', 'Badrinath Temple', 'Balaji Temple Ketkawale', 'Bandel Church', 'Bandra Fort', 'Bandra Worli Sea Link', 'Baneshwar Mandir', 'Bangalore Fort', 'Bangalore Palace', 'Bangalore Town Hall', 'Bankey Bihari Temple', 'Bara Kaman', 'Baramotichi Vihir', 'Baroda Art Museum', 'Baroli Temple Complex', 'Basilica of Bom Jesus', 'Basilica of Our Lady of Dolours', 'Basilica of Our Lady of Graces', 'Basilica of Our Lady of Ransom, Vallarpadam', 'Basilica of Our Lady of Snows, Pallippuram', 'Basilica of Our Lady of the Assumption, Secunderabad', 'Bateshwar Group of Temples', 'Begumpuri Masjid', 'Bekal Fort', 'Belur Math', 'Bhadra Fort', 'Bhairavakona', 'Bhakti Dham Mangarh', 'Bhandasar Jain Temple', 'Bharathi Park', 'Bhavanisagar Dam', 'Bhimas-Ratha', 'Bhimakali Temple', 'Bhimashankar', 'Bhimeshvara Temple Nilagunda', 'Bhoganandishwara Temple', 'Bhojeshwar Temple', 'Bhor Rajwada', 'Bhoramdev Temple', 'Bhuleshwar Temple', 'Bhutanatha group of temples', 'Bidar Fort', 'Biodiversity Park', 'Bir Singh Deo Palace', 'Bir-kangra', 'Birla House Gandhi Smriti', 'Birla Mandir, Hyderabad', 'Birla Mandir, Jaipur', 'Birla Mandir, Kolkata', 'Birla Planetarium', 'Bisaldeo Temple', 'Bnaganga Tank', 'Bogatha Waterfalls', 'Bojjannakonda', 'Bolgatty Palace', 'Bombay Stock Exchane', 'Brahma Temple, Pushkar', 'Brahmesvara Temple', 'Brahmeswara Temple', 'Brihadeeswara Temple', 'Buddha International Circuit', 'Buddha Park of Ravangla', 'Buddha Smriti Park', 'Bull Temple', 'Central Bureau of Investigation', 'Char Minar', 'Chhatrapati Shivaji Terminus', 'City Palace', 'Dagdusheth Halwai Ganpati Temple', 'Dakshineshwar Kali Temple', 'Dakshineswar Kali Temple', 'Dansborg Fort', 'Dargah Aala Hazrat', 'Daria Daulat Bagh', 'Das Mahavidya temple', 'Dashashwamedh Ghat', 'Dassam Falls', 'Dastgeer Sahib', 'Daulatabad, Maharashtra', 'David Sassoon Library', 'Deekshabhoomi', 'Delhi Gate', 'Delhi Town Hall', 'Descent of the Ganges', 'Devanahalli Fort', 'Devi Jagdambi Temple Khajuraho', 'Dhamekh Stupa', 'Dharmaraja Ratha', 'Dharmaraya Swamy Temple', 'Dhom Dam', 'Dhrasanvel Temple, Magderu', 'Dhuandhar Falls', 'Diu Fort', 'Do Drul Chorten', 'Dr MGR Memorial', 'Dras War', 'Dubdi Gompa', 'Dudhsagar Falls', 'DulhaDeo Temple Khajuraho', 'Durga Mandir in Bhelupur', 'Durga Temple, Aihole', 'Dwarkadhish Temple', 'Elephanta Caves', 'Ellora caves', 'Enchey Monastery', 'Eros Cinema', 'Erumbeeswarar Temple', 'Ethipothala Falls', 'Ettumanoor Temple', 'Fatehpur sikri', 'Feroz shah kotla', 'Fort Unchagaon', 'Gateway of India', 'Gobal Vipassana Pagoda', 'Gol Gumbaz', 'Golconda fort', 'Golden Temple', 'Gurudwara Bangla Sahib', 'Gurudwara Sahib, Pushkar', 'Haji ali dargha', 'Hawa Mahal', 'High Court of Karnatka', 'Howrah Bridge', 'Humayu Tomb', 'India gate', 'Jal Mahal', 'Jallianwala Bagh', 'Jama Mashjid', 'Jamia Masjid or Begumpet Mosque', 'Jantar Mantar, Jaipur', 'Jantar Mantar', 'Khajuraho Temple', 'Konark Sun Temple', 'Laxmi Narayan Mandir', 'Lotus Temple', 'Mecca Masjid Hyderabad', 'Mehrangarh Fort', 'Mysore Palace', 'Parliament of India', 'Qutub Minar', 'Rashtrapati Bhavan', 'Red Fort', 'Sajjangarh Monsoon Palace', 'Samudra Narayan Temple', 'Sanchi Stupa', 'Shiv Niwas Palace', 'Shore Temple', 'Shree Banashankari Amma Temple', 'Sidhi Vinayak', 'Supreme Court', 'Taj Mahal', 'The Imperial Tower', 'The Taj Mahal palace hotels', 'Umaid Bhawan Palace', 'Vasai Fort Or Bassien Fort', 'Victoria Memorial', 'Vidhana Soudha']
    classes_logo = ['ACKO', 'Adidas', 'Amazon', 'Amul', 'Aramco', 'BCCI', 'Booking.com', 'Byjus', 'CRED', 'Ceat', 'Chennai Super kings', 'Deccan Chargers', 'Delhi Capitals', 'Domino-s','Dream11', 'EBIX Cash', 'Emirates', 'Fbb','Go Daddy','Gujarat Titans','Gujarat lions','Gulf','Hotstar','IPL','JK lakshmi cement','Jio','Kochi Tuskers','Kolkata Knigth Riders','Lucknow Super Giants','MPL','MRF','Mountain Dew','Mumbai Indians','Myntra','Oppo','Paytm','Pepsi','Postpe','Pune Warriors','Punjab Kings','Rajasthan Royals','Rising Pune Supergiants','Royal Challengers Bangalore','Rupay','Slice IPL','Star Sports','Sunrisers Hyderabad','Swiggy', 'TATA Safari', 'TATA', 'TVS Eurogrip' , 'Unacademy', 'Unicef'    , 'Upstox', 'Vivo', 'safari']
    model_monuments = torch.hub.load('ultralytics/yolov5', 'custom', path='./monument_1.pt')
    model_label = torch.hub.load('ultralytics/yolov5', 'custom', path='./logo.pt')
    video_path = video_path
    # print("video_path "*80)
    # print(video_path)
    cap = cv2.VideoCapture(video_path)

    frame_rate = cap.get(cv2.CAP_PROP_FPS)
    print("frame_rate:",frame_rate)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    monument_results = []
    logo_results = []
    dict_for_monum = {}
    dict_for_logo = {}
    for second in range(int(frame_count / frame_rate)):
        
        print("second:",second)

        start_frame = int(second * frame_rate)
        end_frame = int((second + 1) * frame_rate) - 1

        total_confidence = 0
        monument_names = set()
        logo_names = set()

        for frame_idx in range(start_frame, end_frame + 1):
            ret, frame = cap.read()
            if not ret:
                break

            # Convert the frame to PIL Image
            image = Image.fromarray(frame)
            image.save("./gdgsdfsf.jpg")

            # Perform detection on the frame
            results = model_monuments(image)
            results2 = model_label(image)
            for nested_result_list in results.xywhn:
                for result_tensor in nested_result_list:
                    class_index = int(result_tensor[-1])
                    class_probability = float(result_tensor[-2])
                    class_name = classes_monuments[class_index]
                    total_confidence += class_probability
                    monument_names.add(class_name)
                    print("monuments_name:",monument_names)
        
            for nested_result_list in results2.xywhn:
                for result_tensor in nested_result_list:
                    class_index = int(result_tensor[-1])
                    class_probability = float(result_tensor[-2])
                    class_name = classes_logo[class_index]
                    
                    total_confidence += class_probability
                    logo_names.add(class_name)
                    print("logo_name:",logo_names)
        dict_for_monum[second] = monument_names
        dict_for_logo[second] = logo_names            

        avg_confidence = total_confidence / frame_rate
        timestamp = time.strftime("%H:%M:%S", time.gmtime(second))
        monument_names_str = ', '.join(monument_names)

        avg_confidence = total_confidence / frame_rate
        timestamp = time.strftime("%H:%M:%S", time.gmtime(second))
        logo_names_str = ', '.join(logo_names)
        if monument_names:
            monument_results.append({
                'timestamp': timestamp,
                'monument_names': monument_names_str,
                'confidence': avg_confidence
            })
        if logo_names:
            logo_results.append({
                'timestamp': timestamp,
                'monument_names': logo_names_str,
                'confidence': avg_confidence
            })
    cap.release()

    return monument_results,logo_results
    


In [42]:
monuments,logo=testing_thread_4("D:/Chrome_downloads/video6224143550255204691.mp4")


Using cache found in C:\Users\admin/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-8-7 Python-3.9.13 torch-2.0.1+cpu CPU

Fusing layers... 
custom_YOLOv5m summary: 212 layers, 21685380 parameters, 0 gradients
Adding AutoShape... 
Using cache found in C:\Users\admin/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-8-7 Python-3.9.13 torch-2.0.1+cpu CPU

Fusing layers... 
YOLOv5m summary: 290 layers, 21075189 parameters, 0 gradients
Adding AutoShape... 


frame_rate: 25.0
second: 0
logo_name: {'BCCI'}
logo_name: {'BCCI'}
logo_name: {'BCCI'}
logo_name: {'BCCI'}
logo_name: {'BCCI'}
logo_name: {'BCCI'}
logo_name: {'BCCI'}
logo_name: {'BCCI'}
logo_name: {'BCCI'}
logo_name: {'BCCI'}
logo_name: {'BCCI'}
logo_name: {'BCCI'}
logo_name: {'BCCI'}
logo_name: {'BCCI'}
logo_name: {'BCCI'}
logo_name: {'BCCI'}
logo_name: {'BCCI'}
logo_name: {'BCCI'}
logo_name: {'BCCI'}
logo_name: {'BCCI'}
logo_name: {'BCCI'}
logo_name: {'BCCI'}
logo_name: {'BCCI'}
logo_name: {'BCCI'}
logo_name: {'BCCI'}
second: 1
logo_name: {'BCCI'}
logo_name: {'BCCI'}
logo_name: {'BCCI'}
logo_name: {'BCCI'}
logo_name: {'BCCI'}
logo_name: {'BCCI'}
logo_name: {'BCCI'}
logo_name: {'BCCI'}
logo_name: {'BCCI'}
logo_name: {'BCCI'}
logo_name: {'BCCI'}
logo_name: {'BCCI'}
logo_name: {'BCCI'}
logo_name: {'BCCI'}
logo_name: {'BCCI'}
logo_name: {'BCCI'}
logo_name: {'BCCI'}
logo_name: {'BCCI'}
logo_name: {'BCCI'}
logo_name: {'BCCI'}
logo_name: {'BCCI'}
logo_name: {'BCCI'}
logo_name: {'BCCI'}
log

KeyboardInterrupt: 

In [19]:
def execute_function1(arg):
    result = testing_thread_1(arg)
    return result

In [20]:
def execute_function2(arg):
    result = testing_thread_2(arg)
    return result

In [21]:
def execute_function3(arg):
    result = testing_thread_3(arg)
    return result

In [22]:
def execute_function4(arg):
    result = testing_thread_4(arg)
    return result

In [23]:
app = Flask(__name__)

@app.route('/upload_video', methods=['POST'])
def upload_video():
    try:
        save_path = 'D:/Demo_Resources/Moderation_AI/video/'
        if 'video' not in request.files:
            return 'No video file found', 400

        video_file = request.files['video']

        # Save the video to the specified path
        video_file.save(save_path + video_file.filename)
        path_of_video = (save_path + video_file.filename)
        split_video_into_segments(path_of_video)
        list_video_segments = get_videos_with_nomenclature()

        # path_of_wav_file = convert_video_to_wav(path_of_video)
# =====================================================================================
        threads = []
        results_function1 = []
        results_function2 = []
        results_function3 = []
        results_function4 = []
        function1_args = [list_video_segments,]
        function2_args =  [list_video_segments,]
        function3_args = [video_file,]
        function4_args = [path_of_video,]
        for arg in function2_args:
            thread = threading.Thread(target=lambda arg=arg: results_function2.append(execute_function2 (arg)))
            threads.append(thread)
            thread.start()
        
        for arg in function1_args:
            thread = threading.Thread(target=lambda arg=arg: results_function1.append(execute_function1(arg)))
            threads.append(thread)
            thread.start()
        for arg in function3_args:
            thread = threading.Thread(target=lambda arg=arg: results_function3.append(execute_function3(arg)))
            threads.append(thread)
            thread.start()
        for arg in function4_args:
            thread = threading.Thread(target=lambda arg=arg: results_function4.append(execute_function4(arg)))
            threads.append(thread)
            thread.start()

        # thread = threading.Thread(target=lambda arg=path_of_video: results_function1.append(execute_function1(path_of_video)))
        # threads.append(thread)
        # thread.start()
        
                
        for thread in threads:
            thread.join()
        print("results_function1"*80)
        print("Results:", results_function1)
        result_dict ={}
        result_dict["results_function1"] = results_function1
        result_dict["results_function2"] = results_function2
        result_dict["results_function3"] = results_function3
        result_dict["results_function4"] = results_function4
        # print("results_function2"*80)
        # print("Results:", results_function2)
        print(result_dict)

        return result_dict,200
# =====================================================================================

    except Exception as e:
        # Log the exception for debugging purposes
        app.logger.error(f"Exception occurred: {e}")
        return str(e), 500

if __name__ == '__main__':
    app.run()
    # print("="*100)
    # print(a)
 

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


Moviepy - Building video D:/Demo_Resources/Moderation_AI/video/patal_lok_short_segment_1.mp4.
MoviePy - Writing audio in patal_lok_short_segment_1TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video D:/Demo_Resources/Moderation_AI/video/patal_lok_short_segment_1.mp4



Moviepy - Done !
Moviepy - video ready D:/Demo_Resources/Moderation_AI/video/patal_lok_short_segment_1.mp4
Video splitting completed successfully!
list_video_segments list_video_segments list_video_segments list_video_segments list_video_segments list_video_segments list_video_segments list_video_segments list_video_segments list_video_segments list_video_segments list_video_segments list_video_segments list_video_segments list_video_segments list_video_segments list_video_segments list_video_segments list_video_segments list_video_segments list_video_segments list_video_segments list_video_segments list_video_segments list_video_segments list_video_segments list_video_segments list_video_segments list_video_segments list_video_segments list_video_segments list_video_segments list_video_segments list_video_segments list_video_segments list_video_segments list_video_segments list_video_segments list_video_segments list_video_segments list_video_segments list_video_segments list_video_se

...

1
SessionEventArgs(session_id=ba3da9b1b4d9425e928b0646fab0c476)


Using cache found in C:\Users\admin/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-8-7 Python-3.9.13 torch-2.0.1+cpu CPU

Fusing layers... 
YOLOv5m summary: 290 layers, 21075189 parameters, 0 gradients
Adding AutoShape... 


video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path video_path 
D:/Demo_Resources/Moderation_AI/video/patal_lok_short.mp4
....tep_text_json: [{'Id': '89151cafbf1d4f7991db4330d18dd926'

127.0.0.1 - - [08/Aug/2023 13:43:02] "POST /upload_video HTTP/1.1" 200 -


results_function1results_function1results_function1results_function1results_function1results_function1results_function1results_function1results_function1results_function1results_function1results_function1results_function1results_function1results_function1results_function1results_function1results_function1results_function1results_function1results_function1results_function1results_function1results_function1results_function1results_function1results_function1results_function1results_function1results_function1results_function1results_function1results_function1results_function1results_function1results_function1results_function1results_function1results_function1results_function1results_function1results_function1results_function1results_function1results_function1results_function1results_function1results_function1results_function1results_function1results_function1results_function1results_function1results_function1results_function1results_function1results_function1results_function1results_functi

In [ ]:
['D:/Demo_Resources/Moderation_AI/video/Patal_lok_segment_1.mp4', 'D:/Demo_Resources/Moderation_AI/video/Patal_lok_segment_2.mp4', 'D:/Demo_Resources/Moderation_AI/video/Patal_lok_segment_3.mp4', 'D:/Demo_Resources/Moderation_AI/video/Patal_lok_segment_4.mp4', 'D:/Demo_Resources/Moderation_AI/video/Patal_lok_segment_5.mp4']


In [ ]:
pip install opencv-python

In [ ]:
video_path = "D:/Demo_Resources/Moderation_AI/video/Wion_10_seconds.mp4"
cap = cv2.VideoCapture(video_path)

In [ ]:
cap 

In [ ]:
['D:/Demo_Resources/Moderation_AI/video/output_segment_000.mp4', 'D:/Demo_Resources/Moderation_AI/video/output_segment_000_segment_1.mp4']


In [ ]:
{
    "results_function2": [
        [
            [
                [
                    {
                        "Confidence": 66.58389282226562,
                        "Moderated Label Name": "Graphic Violence Or Gore",
                        "timestamp": 29000
                    },
                    {
                        "Confidence": 66.833984375,
                        "Moderated Label Name": "Violence",
                        "timestamp": 29000
                    }
                ],
                [
                    {
                        "Confidence": 51.089508056640625,
                        "Moderated Label Name": "Barechested Male",
                        "timestamp": 3000
                    },
                    {
                        "Confidence": 51.089508056640625,
                        "Moderated Label Name": "Suggestive",
                        "timestamp": 3000
                    },
                    {
                        "Confidence": 61.03115463256836,
                        "Moderated Label Name": "Barechested Male",
                        "timestamp": 8000
                    },
                    {
                        "Confidence": 61.03115463256836,
                        "Moderated Label Name": "Suggestive",
                        "timestamp": 8000
                    },
                    {
                        "Confidence": 58.915218353271484,
                        "Moderated Label Name": "Barechested Male",
                        "timestamp": 8500
                    },
                    {
                        "Confidence": 58.915218353271484,
                        "Moderated Label Name": "Suggestive",
                        "timestamp": 8500
                    },
                    {
                        "Confidence": 54.14323425292969,
                        "Moderated Label Name": "Barechested Male",
                        "timestamp": 19500
                    },
                    {
                        "Confidence": 54.14323425292969,
                        "Moderated Label Name": "Suggestive",
                        "timestamp": 19500
                    },
                    {
                        "Confidence": 62.74090576171875,
                        "Moderated Label Name": "Revealing Clothes",
                        "timestamp": 31000
                    },
                    {
                        "Confidence": 62.77024841308594,
                        "Moderated Label Name": "Suggestive",
                        "timestamp": 31000
                    },
                    {
                        "Confidence": 65.75260925292969,
                        "Moderated Label Name": "Revealing Clothes",
                        "timestamp": 31500
                    },
                    {
                        "Confidence": 65.75260925292969,
                        "Moderated Label Name": "Suggestive",
                        "timestamp": 31500
                    },
                    {
                        "Confidence": 57.27568817138672,
                        "Moderated Label Name": "Revealing Clothes",
                        "timestamp": 32000
                    },
                    {
                        "Confidence": 57.27568817138672,
                        "Moderated Label Name": "Suggestive",
                        "timestamp": 32000
                    }
                ],
                [
                    {
                        "Confidence": 68.03194427490234,
                        "Moderated Label Name": "Revealing Clothes",
                        "timestamp": 2000
                    },
                    {
                        "Confidence": 68.03384399414062,
                        "Moderated Label Name": "Suggestive",
                        "timestamp": 2000
                    },
                    {
                        "Confidence": 73.57652282714844,
                        "Moderated Label Name": "Revealing Clothes",
                        "timestamp": 2500
                    },
                    {
                        "Confidence": 73.58142852783203,
                        "Moderated Label Name": "Suggestive",
                        "timestamp": 2500
                    },
                    {
                        "Confidence": 85.54639434814453,
                        "Moderated Label Name": "Revealing Clothes",
                        "timestamp": 13000
                    },
                    {
                        "Confidence": 85.55010223388672,
                        "Moderated Label Name": "Suggestive",
                        "timestamp": 13000
                    },
                    {
                        "Confidence": 93.99777221679688,
                        "Moderated Label Name": "Revealing Clothes",
                        "timestamp": 13500
                    },
                    {
                        "Confidence": 93.99896240234375,
                        "Moderated Label Name": "Suggestive",
                        "timestamp": 13500
                    },
                    {
                        "Confidence": 95.06069946289062,
                        "Moderated Label Name": "Revealing Clothes",
                        "timestamp": 14000
                    },
                    {
                        "Confidence": 95.06069946289062,
                        "Moderated Label Name": "Suggestive",
                        "timestamp": 14000
                    },
                    {
                        "Confidence": 84.9251480102539,
                        "Moderated Label Name": "Revealing Clothes",
                        "timestamp": 14500
                    },
                    {
                        "Confidence": 84.92725372314453,
                        "Moderated Label Name": "Suggestive",
                        "timestamp": 14500
                    },
                    {
                        "Confidence": 68.11420440673828,
                        "Moderated Label Name": "Revealing Clothes",
                        "timestamp": 15000
                    },
                    {
                        "Confidence": 68.12122344970703,
                        "Moderated Label Name": "Suggestive",
                        "timestamp": 15000
                    },
                    {
                        "Confidence": 76.71624755859375,
                        "Moderated Label Name": "Explicit Nudity",
                        "timestamp": 32000
                    },
                    {
                        "Confidence": 76.71624755859375,
                        "Moderated Label Name": "Sexual Activity",
                        "timestamp": 32000
                    },
                    {
                        "Confidence": 82.21062469482422,
                        "Moderated Label Name": "Explicit Nudity",
                        "timestamp": 32500
                    },
                    {
                        "Confidence": 82.21062469482422,
                        "Moderated Label Name": "Sexual Activity",
                        "timestamp": 32500
                    },
                    {
                        "Confidence": 51.755367279052734,
                        "Moderated Label Name": "Explicit Nudity",
                        "timestamp": 33000
                    },
                    {
                        "Confidence": 51.755367279052734,
                        "Moderated Label Name": "Sexual Activity",
                        "timestamp": 33000
                    },
                    {
                        "Confidence": 88.44176483154297,
                        "Moderated Label Name": "Explicit Nudity",
                        "timestamp": 33500
                    },
                    {
                        "Confidence": 88.44176483154297,
                        "Moderated Label Name": "Sexual Activity",
                        "timestamp": 33500
                    },
                    {
                        "Confidence": 90.96894073486328,
                        "Moderated Label Name": "Explicit Nudity",
                        "timestamp": 34000
                    },
                    {
                        "Confidence": 90.96894073486328,
                        "Moderated Label Name": "Sexual Activity",
                        "timestamp": 34000
                    },
                    {
                        "Confidence": 92.44217681884766,
                        "Moderated Label Name": "Explicit Nudity",
                        "timestamp": 34500
                    },
                    {
                        "Confidence": 92.44217681884766,
                        "Moderated Label Name": "Sexual Activity",
                        "timestamp": 34500
                    },
                    {
                        "Confidence": 95.3062973022461,
                        "Moderated Label Name": "Explicit Nudity",
                        "timestamp": 35000
                    },
                    {
                        "Confidence": 95.3062973022461,
                        "Moderated Label Name": "Sexual Activity",
                        "timestamp": 35000
                    },
                    {
                        "Confidence": 93.62076568603516,
                        "Moderated Label Name": "Explicit Nudity",
                        "timestamp": 35500
                    },
                    {
                        "Confidence": 93.62076568603516,
                        "Moderated Label Name": "Sexual Activity",
                        "timestamp": 35500
                    },
                    {
                        "Confidence": 87.02074432373047,
                        "Moderated Label Name": "Explicit Nudity",
                        "timestamp": 36000
                    },
                    {
                        "Confidence": 87.02074432373047,
                        "Moderated Label Name": "Sexual Activity",
                        "timestamp": 36000
                    },
                    {
                        "Confidence": 67.66307067871094,
                        "Moderated Label Name": "Explicit Nudity",
                        "timestamp": 37500
                    },
                    {
                        "Confidence": 67.66307067871094,
                        "Moderated Label Name": "Sexual Activity",
                        "timestamp": 37500
                    },
                    {
                        "Confidence": 81.88880920410156,
                        "Moderated Label Name": "Explicit Nudity",
                        "timestamp": 38000
                    },
                    {
                        "Confidence": 81.88880920410156,
                        "Moderated Label Name": "Sexual Activity",
                        "timestamp": 38000
                    },
                    {
                        "Confidence": 68.58763122558594,
                        "Moderated Label Name": "Explicit Nudity",
                        "timestamp": 38500
                    },
                    {
                        "Confidence": 68.58763122558594,
                        "Moderated Label Name": "Sexual Activity",
                        "timestamp": 38500
                    },
                    {
                        "Confidence": 50.47111892700195,
                        "Moderated Label Name": "Sexual Situations",
                        "timestamp": 38500
                    },
                    {
                        "Confidence": 50.73074722290039,
                        "Moderated Label Name": "Suggestive",
                        "timestamp": 38500
                    },
                    {
                        "Confidence": 56.25375747680664,
                        "Moderated Label Name": "Sexual Situations",
                        "timestamp": 39000
                    },
                    {
                        "Confidence": 56.77301025390625,
                        "Moderated Label Name": "Suggestive",
                        "timestamp": 39000
                    },
                    {
                        "Confidence": 50.1187629699707,
                        "Moderated Label Name": "Suggestive",
                        "timestamp": 40000
                    },
                    {
                        "Confidence": 62.17461013793945,
                        "Moderated Label Name": "Explicit Nudity",
                        "timestamp": 40500
                    },
                    {
                        "Confidence": 62.17461013793945,
                        "Moderated Label Name": "Sexual Activity",
                        "timestamp": 40500
                    },
                    {
                        "Confidence": 51.31953048706055,
                        "Moderated Label Name": "Explicit Nudity",
                        "timestamp": 41000
                    },
                    {
                        "Confidence": 51.31953048706055,
                        "Moderated Label Name": "Sexual Activity",
                        "timestamp": 41000
                    },
                    {
                        "Confidence": 78.59529876708984,
                        "Moderated Label Name": "Explicit Nudity",
                        "timestamp": 41500
                    },
                    {
                        "Confidence": 78.59529876708984,
                        "Moderated Label Name": "Sexual Activity",
                        "timestamp": 41500
                    },
                    {
                        "Confidence": 50.939613342285156,
                        "Moderated Label Name": "Sexual Situations",
                        "timestamp": 42000
                    },
                    {
                        "Confidence": 50.939613342285156,
                        "Moderated Label Name": "Suggestive",
                        "timestamp": 42000
                    },
                    {
                        "Confidence": 77.11102294921875,
                        "Moderated Label Name": "Explicit Nudity",
                        "timestamp": 43500
                    },
                    {
                        "Confidence": 77.11102294921875,
                        "Moderated Label Name": "Sexual Activity",
                        "timestamp": 43500
                    },
                    {
                        "Confidence": 59.59571838378906,
                        "Moderated Label Name": "Sexual Situations",
                        "timestamp": 44000
                    },
                    {
                        "Confidence": 59.59571838378906,
                        "Moderated Label Name": "Suggestive",
                        "timestamp": 44000
                    },
                    {
                        "Confidence": 52.45446014404297,
                        "Moderated Label Name": "Explicit Nudity",
                        "timestamp": 44500
                    },
                    {
                        "Confidence": 52.45446014404297,
                        "Moderated Label Name": "Sexual Activity",
                        "timestamp": 44500
                    },
                    {
                        "Confidence": 59.789710998535156,
                        "Moderated Label Name": "Sexual Situations",
                        "timestamp": 44500
                    },
                    {
                        "Confidence": 59.789710998535156,
                        "Moderated Label Name": "Suggestive",
                        "timestamp": 44500
                    },
                    {
                        "Confidence": 73.83021545410156,
                        "Moderated Label Name": "Explicit Nudity",
                        "timestamp": 47000
                    },
                    {
                        "Confidence": 73.83021545410156,
                        "Moderated Label Name": "Sexual Activity",
                        "timestamp": 47000
                    },
                    {
                        "Confidence": 58.68696594238281,
                        "Moderated Label Name": "Sexual Situations",
                        "timestamp": 47000
                    },
                    {
                        "Confidence": 58.68696594238281,
                        "Moderated Label Name": "Suggestive",
                        "timestamp": 47000
                    },
                    {
                        "Confidence": 52.42470932006836,
                        "Moderated Label Name": "Sexual Situations",
                        "timestamp": 47500
                    },
                    {
                        "Confidence": 52.42470932006836,
                        "Moderated Label Name": "Suggestive",
                        "timestamp": 47500
                    },
                    {
                        "Confidence": 79.7386474609375,
                        "Moderated Label Name": "Sexual Situations",
                        "timestamp": 48000
                    },
                    {
                        "Confidence": 79.7386474609375,
                        "Moderated Label Name": "Suggestive",
                        "timestamp": 48000
                    },
                    {
                        "Confidence": 77.69541931152344,
                        "Moderated Label Name": "Sexual Situations",
                        "timestamp": 49000
                    },
                    {
                        "Confidence": 77.69541931152344,
                        "Moderated Label Name": "Suggestive",
                        "timestamp": 49000
                    },
                    {
                        "Confidence": 76.53313446044922,
                        "Moderated Label Name": "Sexual Situations",
                        "timestamp": 49500
                    },
                    {
                        "Confidence": 76.53313446044922,
                        "Moderated Label Name": "Suggestive",
                        "timestamp": 49500
                    },
                    {
                        "Confidence": 63.77994918823242,
                        "Moderated Label Name": "Sexual Situations",
                        "timestamp": 50000
                    },
                    {
                        "Confidence": 63.77994918823242,
                        "Moderated Label Name": "Suggestive",
                        "timestamp": 50000
                    },
                    {
                        "Confidence": 83.47488403320312,
                        "Moderated Label Name": "Explicit Nudity",
                        "timestamp": 51000
                    },
                    {
                        "Confidence": 83.47488403320312,
                        "Moderated Label Name": "Sexual Activity",
                        "timestamp": 51000
                    },
                    {
                        "Confidence": 57.315025329589844,
                        "Moderated Label Name": "Sexual Situations",
                        "timestamp": 51000
                    },
                    {
                        "Confidence": 57.315025329589844,
                        "Moderated Label Name": "Suggestive",
                        "timestamp": 51000
                    },
                    {
                        "Confidence": 91.94335174560547,
                        "Moderated Label Name": "Explicit Nudity",
                        "timestamp": 51500
                    },
                    {
                        "Confidence": 91.94335174560547,
                        "Moderated Label Name": "Sexual Activity",
                        "timestamp": 51500
                    },
                    {
                        "Confidence": 89.42103576660156,
                        "Moderated Label Name": "Explicit Nudity",
                        "timestamp": 52000
                    },
                    {
                        "Confidence": 89.42103576660156,
                        "Moderated Label Name": "Sexual Activity",
                        "timestamp": 52000
                    },
                    {
                        "Confidence": 81.57727813720703,
                        "Moderated Label Name": "Sexual Situations",
                        "timestamp": 52000
                    },
                    {
                        "Confidence": 81.57727813720703,
                        "Moderated Label Name": "Suggestive",
                        "timestamp": 52000
                    },
                    {
                        "Confidence": 85.4544677734375,
                        "Moderated Label Name": "Explicit Nudity",
                        "timestamp": 52500
                    },
                    {
                        "Confidence": 85.4544677734375,
                        "Moderated Label Name": "Sexual Activity",
                        "timestamp": 52500
                    },
                    {
                        "Confidence": 66.5801773071289,
                        "Moderated Label Name": "Sexual Situations",
                        "timestamp": 52500
                    },
                    {
                        "Confidence": 66.5801773071289,
                        "Moderated Label Name": "Suggestive",
                        "timestamp": 52500
                    },
                    {
                        "Confidence": 55.29951477050781,
                        "Moderated Label Name": "Explicit Nudity",
                        "timestamp": 53000
                    },
                    {
                        "Confidence": 55.29951477050781,
                        "Moderated Label Name": "Sexual Activity",
                        "timestamp": 53000
                    },
                    {
                        "Confidence": 84.95277404785156,
                        "Moderated Label Name": "Sexual Situations",
                        "timestamp": 53000
                    },
                    {
                        "Confidence": 84.95277404785156,
                        "Moderated Label Name": "Suggestive",
                        "timestamp": 53000
                    },
                    {
                        "Confidence": 51.3968505859375,
                        "Moderated Label Name": "Explicit Nudity",
                        "timestamp": 53500
                    },
                    {
                        "Confidence": 51.3968505859375,
                        "Moderated Label Name": "Sexual Activity",
                        "timestamp": 53500
                    },
                    {
                        "Confidence": 69.90733337402344,
                        "Moderated Label Name": "Sexual Situations",
                        "timestamp": 53500
                    },
                    {
                        "Confidence": 69.90733337402344,
                        "Moderated Label Name": "Suggestive",
                        "timestamp": 53500
                    },
                    {
                        "Confidence": 69.67269134521484,
                        "Moderated Label Name": "Sexual Situations",
                        "timestamp": 54000
                    },
                    {
                        "Confidence": 69.67269134521484,
                        "Moderated Label Name": "Suggestive",
                        "timestamp": 54000
                    },
                    {
                        "Confidence": 85.32067108154297,
                        "Moderated Label Name": "Explicit Nudity",
                        "timestamp": 54500
                    },
                    {
                        "Confidence": 85.32067108154297,
                        "Moderated Label Name": "Sexual Activity",
                        "timestamp": 54500
                    },
                    {
                        "Confidence": 76.56755828857422,
                        "Moderated Label Name": "Sexual Situations",
                        "timestamp": 54500
                    }
                ],
                [
                    {
                        "Confidence": 76.9449691772461,
                        "Moderated Label Name": "Explicit Nudity",
                        "timestamp": 2000
                    },
                    {
                        "Confidence": 76.9449691772461,
                        "Moderated Label Name": "Sexual Activity",
                        "timestamp": 2000
                    },
                    {
                        "Confidence": 73.1579360961914,
                        "Moderated Label Name": "Sexual Situations",
                        "timestamp": 2000
                    },
                    {
                        "Confidence": 73.1579360961914,
                        "Moderated Label Name": "Suggestive",
                        "timestamp": 2000
                    },
                    {
                        "Confidence": 63.24049758911133,
                        "Moderated Label Name": "Explicit Nudity",
                        "timestamp": 2500
                    },
                    {
                        "Confidence": 63.24049758911133,
                        "Moderated Label Name": "Sexual Activity",
                        "timestamp": 2500
                    },
                    {
                        "Confidence": 83.90328216552734,
                        "Moderated Label Name": "Sexual Situations",
                        "timestamp": 2500
                    },
                    {
                        "Confidence": 83.90328216552734,
                        "Moderated Label Name": "Suggestive",
                        "timestamp": 2500
                    },
                    {
                        "Confidence": 77.51277160644531,
                        "Moderated Label Name": "Explicit Nudity",
                        "timestamp": 3000
                    },
                    {
                        "Confidence": 77.51277160644531,
                        "Moderated Label Name": "Sexual Activity",
                        "timestamp": 3000
                    },
                    {
                        "Confidence": 72.38484191894531,
                        "Moderated Label Name": "Sexual Situations",
                        "timestamp": 3000
                    },
                    {
                        "Confidence": 72.38484191894531,
                        "Moderated Label Name": "Suggestive",
                        "timestamp": 3000
                    },
                    {
                        "Confidence": 54.47235870361328,
                        "Moderated Label Name": "Violence",
                        "timestamp": 17500
                    },
                    {
                        "Confidence": 54.47114944458008,
                        "Moderated Label Name": "Weapons",
                        "timestamp": 17500
                    },
                    {
                        "Confidence": 63.817508697509766,
                        "Moderated Label Name": "Violence",
                        "timestamp": 18000
                    },
                    {
                        "Confidence": 63.81719207763672,
                        "Moderated Label Name": "Weapons",
                        "timestamp": 18000
                    },
                    {
                        "Confidence": 70.78406524658203,
                        "Moderated Label Name": "Violence",
                        "timestamp": 19500
                    },
                    {
                        "Confidence": 70.78406524658203,
                        "Moderated Label Name": "Weapons",
                        "timestamp": 19500
                    },
                    {
                        "Confidence": 65.57862091064453,
                        "Moderated Label Name": "Violence",
                        "timestamp": 20000
                    },
                    {
                        "Confidence": 65.57862091064453,
                        "Moderated Label Name": "Weapons",
                        "timestamp": 20000
                    },
                    {
                        "Confidence": 52.007164001464844,
                        "Moderated Label Name": "Violence",
                        "timestamp": 20500
                    },
                    {
                        "Confidence": 52.007164001464844,
                        "Moderated Label Name": "Weapons",
                        "timestamp": 20500
                    },
                    {
                        "Confidence": 53.18218231201172,
                        "Moderated Label Name": "Violence",
                        "timestamp": 21000
                    },
                    {
                        "Confidence": 53.18218231201172,
                        "Moderated Label Name": "Weapons",
                        "timestamp": 21000
                    },
                    {
                        "Confidence": 65.41690826416016,
                        "Moderated Label Name": "Violence",
                        "timestamp": 22000
                    },
                    {
                        "Confidence": 65.41690826416016,
                        "Moderated Label Name": "Weapons",
                        "timestamp": 22000
                    },
                    {
                        "Confidence": 57.819007873535156,
                        "Moderated Label Name": "Violence",
                        "timestamp": 22500
                    },
                    {
                        "Confidence": 57.819007873535156,
                        "Moderated Label Name": "Weapons",
                        "timestamp": 22500
                    },
                    {
                        "Confidence": 69.71326446533203,
                        "Moderated Label Name": "Violence",
                        "timestamp": 23000
                    },
                    {
                        "Confidence": 69.71326446533203,
                        "Moderated Label Name": "Weapons",
                        "timestamp": 23000
                    },
                    {
                        "Confidence": 78.62223815917969,
                        "Moderated Label Name": "Violence",
                        "timestamp": 30500
                    },
                    {
                        "Confidence": 78.54130554199219,
                        "Moderated Label Name": "Weapons",
                        "timestamp": 30500
                    },
                    {
                        "Confidence": 67.99032592773438,
                        "Moderated Label Name": "Violence",
                        "timestamp": 31000
                    },
                    {
                        "Confidence": 67.82846069335938,
                        "Moderated Label Name": "Weapons",
                        "timestamp": 31000
                    },
                    {
                        "Confidence": 79.27029418945312,
                        "Moderated Label Name": "Violence",
                        "timestamp": 33000
                    },
                    {
                        "Confidence": 79.27029418945312,
                        "Moderated Label Name": "Weapons",
                        "timestamp": 33000
                    },
                    {
                        "Confidence": 83.16478729248047,
                        "Moderated Label Name": "Violence",
                        "timestamp": 33500
                    },
                    {
                        "Confidence": 83.16478729248047,
                        "Moderated Label Name": "Weapons",
                        "timestamp": 33500
                    },
                    {
                        "Confidence": 87.9580078125,
                        "Moderated Label Name": "Violence",
                        "timestamp": 34000
                    },
                    {
                        "Confidence": 59.4473876953125,
                        "Moderated Label Name": "Weapon Violence",
                        "timestamp": 34000
                    },
                    {
                        "Confidence": 87.9580078125,
                        "Moderated Label Name": "Weapons",
                        "timestamp": 34000
                    },
                    {
                        "Confidence": 67.99642944335938,
                        "Moderated Label Name": "Violence",
                        "timestamp": 34500
                    },
                    {
                        "Confidence": 54.631385803222656,
                        "Moderated Label Name": "Weapon Violence",
                        "timestamp": 34500
                    },
                    {
                        "Confidence": 67.99642944335938,
                        "Moderated Label Name": "Weapons",
                        "timestamp": 34500
                    },
                    {
                        "Confidence": 88.66431427001953,
                        "Moderated Label Name": "Violence",
                        "timestamp": 35000
                    },
                    {
                        "Confidence": 62.29255676269531,
                        "Moderated Label Name": "Weapon Violence",
                        "timestamp": 35000
                    },
                    {
                        "Confidence": 88.66431427001953,
                        "Moderated Label Name": "Weapons",
                        "timestamp": 35000
                    },
                    {
                        "Confidence": 89.81148529052734,
                        "Moderated Label Name": "Violence",
                        "timestamp": 35500
                    },
                    {
                        "Confidence": 64.09835052490234,
                        "Moderated Label Name": "Weapon Violence",
                        "timestamp": 35500
                    },
                    {
                        "Confidence": 89.81148529052734,
                        "Moderated Label Name": "Weapons",
                        "timestamp": 35500
                    },
                    {
                        "Confidence": 91.45232391357422,
                        "Moderated Label Name": "Violence",
                        "timestamp": 36000
                    },
                    {
                        "Confidence": 64.25054168701172,
                        "Moderated Label Name": "Weapon Violence",
                        "timestamp": 36000
                    },
                    {
                        "Confidence": 91.45232391357422,
                        "Moderated Label Name": "Weapons",
                        "timestamp": 36000
                    },
                    {
                        "Confidence": 91.32391357421875,
                        "Moderated Label Name": "Violence",
                        "timestamp": 36500
                    },
                    {
                        "Confidence": 63.57738494873047,
                        "Moderated Label Name": "Weapon Violence",
                        "timestamp": 36500
                    },
                    {
                        "Confidence": 91.32391357421875,
                        "Moderated Label Name": "Weapons",
                        "timestamp": 36500
                    },
                    {
                        "Confidence": 91.25582122802734,
                        "Moderated Label Name": "Violence",
                        "timestamp": 37000
                    },
                    {
                        "Confidence": 63.05241775512695,
                        "Moderated Label Name": "Weapon Violence",
                        "timestamp": 37000
                    },
                    {
                        "Confidence": 91.25582122802734,
                        "Moderated Label Name": "Weapons",
                        "timestamp": 37000
                    },
                    {
                        "Confidence": 86.98414611816406,
                        "Moderated Label Name": "Violence",
                        "timestamp": 37500
                    },
                    {
                        "Confidence": 59.17583084106445,
                        "Moderated Label Name": "Weapon Violence",
                        "timestamp": 37500
                    },
                    {
                        "Confidence": 86.98408508300781,
                        "Moderated Label Name": "Weapons",
                        "timestamp": 37500
                    },
                    {
                        "Confidence": 78.56197357177734,
                        "Moderated Label Name": "Violence",
                        "timestamp": 38000
                    },
                    {
                        "Confidence": 55.16143035888672,
                        "Moderated Label Name": "Weapon Violence",
                        "timestamp": 38000
                    },
                    {
                        "Confidence": 78.56128692626953,
                        "Moderated Label Name": "Weapons",
                        "timestamp": 38000
                    },
                    {
                        "Confidence": 70.10709381103516,
                        "Moderated Label Name": "Smoking",
                        "timestamp": 55000
                    },
                    {
                        "Confidence": 70.10709381103516,
                        "Moderated Label Name": "Tobacco",
                        "timestamp": 55000
                    }
                ],
                []
            ],
            200
        ]
    ]
}